In [ ]:
import nest_asyncio
import ast
from dhanhq import dhanhq, DhanContext, MarketFeed
import functions as f
import pandas as pd
import numpy as np
from config import MYPATHS


In [ ]:
database_path = MYPATHS['data'] + "\\database.txt"
client_id = str(ast.literal_eval(f.get_line(database_path, 3).strip())['client_id'])
access_token = str(ast.literal_eval(f.get_line(database_path, 4).strip())['access_token'])

# Correct v2.1.0 initialization
dhan_context = DhanContext(client_id, access_token)
dhan = dhanhq(dhan_context)


In [ ]:
def place_nifty_bracket_order(quantity=50, entry_price=100.0, target_price=120.0, stoploss_price=90.0):
    """
    Place a Bracket order for NIFTY options
    
    Args:
        quantity: Number of lots/shares to trade
        entry_price: Entry price for the order
        target_price: Target price (absolute price, not points)
        stoploss_price: Stop loss price (absolute price, not points)
    """

    # Method 2: Fallback to regular BO if super order fails
    try:
        print("Trying regular bracket order...")
        response = dhan.place_order(
            security_id='50990',
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.BUY,
            quantity=quantity,
            order_type=dhan.LIMIT,
            product_type=dhan.BO,
            price=entry_price,
            bo_profit_value=target_price - entry_price,
            bo_stop_loss_Value=entry_price - stoploss_price,
            validity=dhan.DAY,
            tag="nifty_bracket_001"
        )
        
        print("Bracket Order Response:", response)
        return response
        
    except Exception as e2:
        print(f"Error placing bracket order: {e2}")
        return None

# Example usage
if __name__ == "__main__":
    # Place super order with absolute prices
    order_response = place_nifty_bracket_order(
        quantity=75,           # 50 shares/lots
        entry_price=121.0,     # Entry at ₹100
        target_price=130.0,    # Target at ₹120 (absolute price)
        stoploss_price=90.0    # Stop loss at ₹90 (absolute price)
    )
    
    # Check order status
    if order_response and order_response.get('status') == 'success':
        order_id = order_response.get('data', {}).get('orderId')
        print(f"Bracket order placed successfully. Order ID: {order_id}")
    else:
        print("Failed to place bracket order")

In [ ]:
order_response = {'status': 'success',
    'remarks': '',
    'data': {'orderId': '92250613413726', 'orderStatus': 'TRANSIT'}}
order_response